# DatabricksのABAC実装方法
[属性ベースのアクセス制御（ABAC）](https://techblog.ap-com.co.jp/entry/2025/08/18/142244)<br>
サーバレスコンピュートをお使いください

In [0]:
%run ./00_config

### 1. データ読込み

`/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/daily_weather_tokyo.csv`から<br>
read_filesを利用してcsvを読み込んで表示する。ただし、_rescued_dataは除外する。1行目は列名。

In [0]:
# # 従業員プロファイルテーブルを作成
# spark.sql(f"""
# CREATE TABLE IF NOT EXISTS employees (
#     employee_id INT,
#     name STRING,
#     email STRING,
#     department STRING,
#     salary DOUBLE,
#     region STRING
# )
# USING DELTA;
# """)

# # データを挿入
# spark.sql(f"""
# INSERT INTO employees (employee_id, name, email, department, salary, region)
# VALUES
# (1, 'Alice Johnson', 'alice.j@example.com', 'Sales', 75000.00, 'APAC'),
# (2, 'Bob Williams', 'bob.w@example.com', 'Marketing', 68000.00, 'EMEA'),
# (3, 'Charlie Brown', 'charlie.b@example.com', 'Engineering', 92000.00, 'Americas'),
# (4, 'Diana Prince', 'diana.p@example.com', 'HR', 60000.00, 'APAC'),
# (5, 'Eve Davis', 'eve.d@example.com', 'Finance', 80000.00, 'EMEA'),
# (6, 'Frank White', 'frank.w@example.com', 'Sales', 70000.00, 'Americas'),
# (7, 'Grace Lee', 'grace.l@example.com', 'Engineering', 95000.00, 'APAC'),
# (8, 'Henry Kim', 'henry.k@example.com', 'Marketing', 65000.00, 'EMEA'),
# (9, 'Ivy Chen', 'ivy.c@example.com', 'HR', 58000.00, 'Americas'),
# (10, 'Jack Green', 'jack.g@example.com', 'Finance', 82000.00, 'APAC');
# """)

# # ---------- CSV 出力 ----------
# out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/employees.csv"
# spark.table("employees").coalesce(1).toPandas().to_csv(out_path, index=False)
# print(out_path)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {MY_CATALOG}.{MY_SCHEMA}.employees
SELECT *
EXCEPT (_rescued_data)
FROM read_files(
  '/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/employees.csv',
  format => 'csv',
  header => 'true'
)
;
""")

display(spark.table("employees"))

In [0]:
# spark.sql(f"""
# drop table if exists employees
# """)

### 2. 管理タグ作成
カタログ -> 管理タグ 

<img src='https://github.com/komae5519pv/databricks_quick_demo/blob/main/20250912_quick_abac/images/%E7%AE%A1%E7%90%86%E3%82%BF%E3%82%AF%E3%82%99%E4%BD%9C%E6%88%90.gif?raw=true' width='88%'/>


### 3. 列へのタグ設定

In [0]:
# 'region' 列にアクセス制限タグを設定
spark.sql(f"""
ALTER TABLE {MY_CATALOG}.{MY_SCHEMA}.employees
ALTER COLUMN email SET TAGS ('my_tags' = 'pii_email')
""")

In [0]:
# 'region' 列にアクセス制限タグを設定
spark.sql(f"""
ALTER TABLE {MY_CATALOG}.{MY_SCHEMA}.employees
ALTER COLUMN salary SET TAGS ('my_tags' = 'salary')
""")

In [0]:
# 'region' 列にアクセス制限タグを設定
spark.sql(f"""
ALTER TABLE {MY_CATALOG}.{MY_SCHEMA}.employees
ALTER COLUMN region SET TAGS ('my_tags' = 'region_apac')
""")

### 4. 列マスキングポリシーで使用するUDFの作成
個人情報をマスクするロジックは事前に関数としてUC登録しておく必要があります。

In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {MY_CATALOG}.{MY_SCHEMA}.mask_email(email STRING)
RETURNS STRING
LANGUAGE PYTHON
AS $$
parts = email.split('@', 1)
if len(parts) == 2:
  username, domain = parts
else:
  return None
masked_username = username[0] + '*' * (len(username) - 2) + username[-1]
return masked_username + "@" + domain
$$
""")

In [0]:
# spark.sql(f"""
# DROP FUNCTION IF EXISTS {MY_CATALOG}.{MY_SCHEMA}.mask_email
# """)

In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {MY_CATALOG}.{MY_SCHEMA}.mask_salary(val DOUBLE)
RETURNS DOUBLE
LANGUAGE PYTHON
AS $$
return -99.99
$$
""")

In [0]:
# spark.sql(f"""
# DROP FUNCTION IF EXISTS {MY_CATALOG}.{MY_SCHEMA}.mask_salary
# """)

### 5. 行フィルターポリシーで使用するUDFの作成

In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {MY_CATALOG}.{MY_SCHEMA}.is_apac_region(employee_region STRING)
RETURNS BOOLEAN
RETURN (
    SELECT CASE
        WHEN employee_region = 'APAC' THEN TRUE
        ELSE FALSE
    END
)
""")

In [0]:
# spark.sql(f"""
# DROP FUNCTION IF EXISTS {MY_CATALOG}.{MY_SCHEMA}.is_apac_region
# """)

### 6. カタログエクスプローラーでのポリシー設定
カタログ -> ポリシー

<img src='https://github.com/komae5519pv/databricks_quick_demo/blob/main/20250912_quick_abac/images/%E3%83%9B%E3%82%9A%E3%83%AA%E3%82%B7%E3%83%BC%E8%A8%AD%E5%AE%9A.gif?raw=true' width='88%'/>
